In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
import gc
from tqdm import tqdm
from warnings import filterwarnings
filterwarnings('ignore')

%matplotlib inline

PATH = '../'
RND_SEED = 44

In [5]:
%%time
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'Track 1/test.csv')
submission = pd.read_csv(PATH + 'Track 1/submission.csv')

Wall time: 21.2 s


### LB Score Calc

Для подсчета скора просто используйте функцию getLBScore

In [6]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs(satellite_predicted_values - satellite_true_values) / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values)))

### Model training

In [7]:
training_columns = ['epoch', 'sat_id', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']
prediction_columns = ['x','y','z','Vx', 'Vy', 'Vz']
X = train.loc[:, training_columns]
y = train.loc[:, prediction_columns]
Xtest = test.loc[:, training_columns]

In [8]:
X.drop(columns = 'epoch', inplace = True)
Xtest.drop(columns = 'epoch', inplace = True)

In [1]:
shiftedSat = [0,
 1,
 2,
 3,
 4,
 7,
 10,
 12,
 13,
 15,
 17,
 18,
 19,
 21,
 23,
 25,
 27,
 32,
 33,
 37,
 38,
 41,
 45,
 48,
 50,
 52,
 59,
 62,
 63,
 65,
 66,
 70,
 73,
 80,
 82,
 84,
 85,
 86,
 88,
 89,
 90,
 92,
 94,
 98,
 99,
 100,
 101,
 109,
 112,
 118,
 123,
 124,
 125,
 126,
 127,
 133,
 134,
 136,
 141,
 149,
 150,
 151,
 155,
 166,
 167,
 173,
 174,
 175,
 177,
 184,
 186,
 188,
 190,
 195,
 202,
 204,
 206,
 208,
 209,
 213,
 217,
 223,
 230,
 236,
 241,
 242,
 243,
 244,
 246,
 247,
 248,
 249,
 251,
 255,
 256,
 257,
 258,
 264,
 266,
 270,
 272,
 274,
 275,
 278,
 280,
 282,
 283,
 286,
 290,
 291,
 308,
 315,
 318,
 320,
 323,
 325,
 328,
 329,
 334,
 340,
 348,
 351,
 361,
 363,
 364,
 365,
 366,
 368,
 371,
 372,
 375,
 376,
 380,
 381,
 382,
 385,
 390,
 392,
 406,
 407,
 409,
 412,
 415,
 416,
 418,
 420,
 421,
 422,
 429,
 437,
 439,
 440,
 441,
 450,
 451,
 452,
 454,
 461,
 464,
 468,
 470,
 472,
 473,
 476,
 477,
 478,
 480,
 482,
 483,
 486,
 490,
 491,
 493,
 494,
 495,
 498,
 500,
 501,
 502,
 503,
 505,
 508,
 509,
 511,
 512,
 514,
 518,
 521,
 523,
 527,
 529,
 530,
 531,
 533,
 535,
 536,
 538,
 539,
 541,
 542,
 546,
 548,
 549,
 550,
 552,
 555,
 558,
 559,
 560,
 565,
 568,
 569,
 575,
 576,
 577,
 580,
 581,
 584,
 585,
 587,
 588,
 590,
 592,
 595,
 596,
 597,
 598]

In [9]:
linearSat = [5,
 6,
 8,
 9,
 11,
 14,
 20,
 22,
 24,
 26,
 28,
 29,
 30,
 31,
 34,
 35,
 36,
 39,
 40,
 42,
 43,
 44,
 46,
 47,
 49,
 51,
 53,
 54,
 55,
 57,
 58,
 60,
 61,
 64,
 67,
 68,
 69,
 71,
 72,
 74,
 75,
 76,
 79,
 81,
 83,
 91,
 93,
 97,
 102,
 103,
 104,
 106,
 107,
 108,
 111,
 113,
 114,
 115,
 116,
 119,
 120,
 121,
 122,
 128,
 129,
 130,
 131,
 132,
 135,
 137,
 138,
 139,
 140,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 152,
 154,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 168,
 170,
 171,
 172,
 176,
 179,
 180,
 183,
 187,
 191,
 193,
 196,
 198,
 200,
 203,
 207,
 210,
 211,
 212,
 214,
 215,
 216,
 218,
 220,
 221,
 222,
 224,
 225,
 226,
 227,
 229,
 231,
 232,
 233,
 234,
 235,
 237,
 238,
 239,
 240,
 245,
 250,
 252,
 253,
 254,
 259,
 260,
 261,
 262,
 263,
 265,
 268,
 271,
 273,
 276,
 277,
 279,
 281,
 284,
 285,
 287,
 289,
 292,
 293,
 294,
 295,
 296,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 305,
 306,
 307,
 309,
 310,
 311,
 312,
 313,
 314,
 316,
 317,
 319,
 321,
 322,
 324,
 326,
 330,
 331,
 332,
 333,
 335,
 337,
 339,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 350,
 352,
 353,
 355,
 356,
 358,
 360,
 362,
 367,
 369,
 370,
 373,
 374,
 377,
 378,
 379,
 383,
 384,
 386,
 387,
 388,
 389,
 391,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 400,
 401,
 403,
 404,
 405,
 408,
 410,
 411,
 413,
 414,
 417,
 419,
 423,
 424,
 425,
 426,
 428,
 430,
 431,
 432,
 433,
 435,
 436,
 438,
 442,
 443,
 444,
 446,
 447,
 448,
 455,
 456,
 458,
 459,
 460,
 462,
 463,
 465,
 466,
 467,
 469,
 471,
 474,
 479,
 485,
 487,
 488,
 489,
 492,
 496,
 499,
 504,
 507,
 510,
 515,
 516,
 517,
 519,
 520,
 522,
 524,
 525,
 526,
 528,
 532,
 534,
 537,
 540,
 543,
 544,
 545,
 547,
 553,
 556,
 557,
 562,
 563,
 564,
 566,
 567,
 570,
 571,
 572,
 573,
 574,
 578,
 579,
 582,
 583,
 586,
 589,
 591,
 593,
 594,
 599]

In [ ]:
shiftedSat = [0,
 1,
 2,
 3,
 4,
 7,
 10,
 12,
 13,
 15,
 17,
 18,
 19,
 21,
 23,
 25,
 27,
 32,
 33,
 37,
 38,
 41,
 45,
 48,
 50,
 52,
 59,
 62,
 63,
 65,
 66,
 70,
 73,
 80,
 82,
 84,
 85,
 86,
 88,
 89,
 90,
 92,
 94,
 98,
 99,
 100,
 101,
 109,
 112,
 118,
 123,
 124,
 125,
 126,
 127,
 133,
 134,
 136,
 141,
 149,
 150,
 151,
 155,
 166,
 167,
 173,
 174,
 175,
 177,
 184,
 186,
 188,
 190,
 195,
 202,
 204,
 206,
 208,
 209,
 213,
 217,
 223,
 230,
 236,
 241,
 242,
 243,
 244,
 246,
 247,
 248,
 249,
 251,
 255,
 256,
 257,
 258,
 264,
 266,
 270,
 272,
 274,
 275,
 278,
 280,
 282,
 283,
 286,
 290,
 291,
 308,
 315,
 318,
 320,
 323,
 325,
 328,
 329,
 334,
 340,
 348,
 351,
 361,
 363,
 364,
 365,
 366,
 368,
 371,
 372,
 375,
 376,
 380,
 381,
 382,
 385,
 390,
 392,
 406,
 407,
 409,
 412,
 415,
 416,
 418,
 420,
 421,
 422,
 429,
 437,
 439,
 440,
 441,
 450,
 451,
 452,
 454,
 461,
 464,
 468,
 470,
 472,
 473,
 476,
 477,
 478,
 480,
 482,
 483,
 486,
 490,
 491,
 493,
 494,
 495,
 498,
 500,
 501,
 502,
 503,
 505,
 508,
 509,
 511,
 512,
 514,
 518,
 521,
 523,
 527,
 529,
 530,
 531,
 533,
 535,
 536,
 538,
 539,
 541,
 542,
 546,
 548,
 549,
 550,
 552,
 555,
 558,
 559,
 560,
 565,
 568,
 569,
 575,
 576,
 577,
 580,
 581,
 584,
 585,
 587,
 588,
 590,
 592,
 595,
 596,
 597,
 598]


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
model = LinearRegression()
cluster = KMeans(n_clusters=24, tol = 1e-3, algorithm = 'elkan', random_state=RND_SEED)
scores = []
shift = 0.1
trainSize = 0.6
mult = 2
times = np.arange(0, 1-(mult-1)*trainSize, shift)
for sz,_ in enumerate(times):
    submissionValid = train.copy()[['id', 'sat_id', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz']]
    smp = []
    SMP = []
    nTargets = 6
    for sat_id in tqdm(X['sat_id'].unique()):
        if sat_id not in linearSat:
            smp = []
            satX = X[X['sat_id'] == sat_id]
            satY = y[X['sat_id'] == sat_id]
            
            start = int(sz*(satX.shape[0] * shift))
            size = int(satX.shape[0] * trainSize)
            Xtr, Xval = satX.iloc[start:start+size, :], satX.iloc[start+size:, :]
            ytr, yval = satY.iloc[start:start+size, :], satY.iloc[start+size:, :]
            for i in range(nTargets):
                model.fit(Xtr, ytr.iloc[:, i])
                ypred = model.predict(Xval)
                smp.append(smape(ypred, yval.iloc[:, i]))
                SMP.append(smape(ypred, yval.iloc[:, i]))
        else:
            satX = X[X['sat_id'] == sat_id].drop(columns = ['sat_id'])
            satY = y[X['sat_id'] == sat_id]
            satXtest = Xtest[Xtest['sat_id'] == sat_id].drop(columns = ['sat_id'])

            start = int(sz*(satX.shape[0] * shift))
            size = int(satX.shape[0] * trainSize)
            Xtr, Xval = satX.iloc[start:start+size, :], satX.iloc[start+size:, :]
            ytr, yval = satY.iloc[start:start+size, :], satY.iloc[start+size:, :]

            labelsTrain = cluster.fit_predict(Xtr)
            labelsTest = cluster.predict(Xval)
            smp = []
            for label in np.unique(labelsTest):
                for i in range(nTargets):
                    model.fit(Xtr[labelsTrain == label], ytr[labelsTrain == label].iloc[:, i])
                    ypred = model.predict(Xval[labelsTest == label])
                    smp.append(smape(ypred, yval[labelsTest == label].iloc[:, i]))
                    SMP.append(smape(ypred, yval[labelsTest == label].iloc[:, i]))
    print(f'Train {sz} Final', 100*(1- np.mean(SMP)))
    scores.append(100*(1- np.mean(SMP)))
print(f'Mean fold LB score {np.mean(scores)}')
pd.Series(scores).plot()

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [04:23<00:00,  2.27it/s]


Train 0 Final 93.92111360975257


 32%|█████████████████████████▌                                                      | 192/600 [01:34<02:19,  2.93it/s]

In [141]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import AdaBoostRegressor
model = LinearRegression()
nTargets = 6
for sat_id in tqdm(Xtest['sat_id'].unique()):
    satX = X[X['sat_id'] == sat_id].drop(columns = ['sat_id'])
    satX = pd.concat([satX, satX.shift(1).fillna(0), 
                  satX.shift(2).fillna(0), satX.shift(3).fillna(0),
                 satX.shift(4).fillna(0),
                     satX.shift(5).fillna(0),
                     satX.shift(6).fillna(0)], axis = 1)
    satY = y[X['sat_id'] == sat_id]
    satXtest = Xtest[Xtest['sat_id'] == sat_id].drop(columns = ['sat_id'])
    satXtest = pd.concat([satXtest, satXtest.shift(1).fillna(0),satXtest.shift(2).fillna(0), satXtest.shift(3).fillna(0),
                         satXtest.shift(4).fillna(0),
                         satXtest.shift(5).fillna(0),
                         satXtest.shift(6).fillna(0)], axis = 1)
    
    for i in range(nTargets):
        model.fit(satX, satY.iloc[:, i])
        ypred = model.predict(satXtest)
        submission.loc[satXtest.index, submission.columns[i+1]] = ypred

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:14<00:00, 18.01it/s]


In [143]:
submission.to_csv('submission.csv', index = None)